Libraries

In [ ]:
from pathlib import Path
import re
import statistics as stats
import pandas as pd
from nltk.tokenize import word_tokenize

1. Preprocessing Fairy Tale Corpus | Description | Ranked by Token Count

In [64]:
CORPUS_DIR = Path(r"C:\Users\Sophia\Downloads\MA\CORPORA\Corpus_German FFT")

QUOTE_MAP = str.maketrans({
    '«': '"', '»': '"',  
    '„': '"', '“': '"', '”': '"', 
    '‹': '"', '›': '"',  
})

for fp in CORPUS_DIR.rglob("*.txt"):
    text = fp.read_text(encoding="utf-8")
    new_text = text.translate(QUOTE_MAP)
    if new_text != text:
        fp.write_text(new_text, encoding="utf-8")
print(f"Geänderte Dateien: {changed_files}, Ersetzungen gesamt: {total_replacements}")

Geänderte Dateien: 13, Ersetzungen gesamt: 2916


In [32]:
CORPUS_DIR = Path(r"C:\Users\Sophia\Downloads\MA\CORPORA\Corpus_German FFT")

def count_tokens_and_words(text: str):
    toks = word_tokenize(text)
    token_count = len(toks)
    word_count = sum(bool(word_like.search(t)) and ('_' not in t) for t in toks)
    return token_count, word_count

rows = []
for fp in sorted(CORPUS_DIR.glob("*.txt")):
    txt = fp.read_text(encoding="utf-8")
    tok_c, w_c = count_tokens_and_words(txt)
    rows.append({"File": fp.name, "Tokens": tok_c, "Words": w_c})

df2_all = (
    pd.DataFrame(rows)
      .sort_values("Tokens", ascending=False, ignore_index=True)
      .assign(Rank=lambda d: d.index + 1)
)

#table
df2_view = pd.concat([df2_all.head(3), df2_all.tail(3)], ignore_index=False)[["Rank","File","Tokens","Words"]]
display(df2_view)

,Rank,File,Tokens,Words
0,1,Grimm_Die zwei Brüder_303.txt,9309,7717
1,2,Grimm_Die beiden Wanderer_613.txt,4538,3795
2,3,"Grimm_Märchen vom einem, der auszog das Fürcht...",4364,3429
120,121,Grimm_Die Wassernixe_316.txt,301,259
121,122,Grimm_Der süße Brei_565.txt,272,223
122,123,Grimm_Räthselmärchen_407.txt,139,119


In [34]:
df2_view.to_csv(r"C:\Users\Sophia\Downloads\FFT_Corpus_Size.csv",index=False, encoding="utf-8-sig")

2. Preprocessing AI corpora | Identification quotation marks | Replacing quotation marks

In [ ]:
CORPORA = [
    Path(r"C:\Users\Sophia\Downloads\MA\CORPORA\GPT-4o"),
    Path(r"C:\Users\Sophia\Downloads\MA\CORPORA\GPT-5"),
]

for corpus in CORPORA:
    print(f"\n=== Searching in: {corpus} ===")
    for fp in corpus.rglob("*.txt"):
        text = fp.read_text(encoding="utf-8", errors="ignore")
        for i, line in enumerate(text.splitlines(), start=1):
            if '"' in line:
                print(f"{fp.name} (line {i}): {line}")


=== Searching in: C:\Users\Sophia\Downloads\MA\CORPORA\GPT-4o ===
ChatGPT-4o_1.txt (line 5): "Gib einem müden Wanderer Obdach für die Nacht", sprach er.
ChatGPT-4o_1.txt (line 8): Als er Lenka bemerkte, lächelte er. "Du hast ein reines Herz, Kind. Dafür sollst du einen Lohn erhalten."
ChatGPT-4o_1.txt (line 9): Er reichte ihr den Zweig. "Pflanze ihn an einem Ort, wo nie zuvor eine Pflanze wuchs, und hüte ihn gut. Was daraus wächst, wird dir helfen, wenn deine Stunde der Not kommt."
ChatGPT-4o_1.txt (line 15): "Hier gibt es nichts von Wert", sprach er.
ChatGPT-4o_1.txt (line 17): "Nehmt sie", befahl er.
ChatGPT-4o_1.txt (line 18): Lenka stellte sich schützend davor. "Sie gehört mir."
ChatGPT-4o_1.txt (line 24): "Du hast deine Gabe verloren, doch nicht umsonst. Wenn das Herz rein bleibt, wächst auch aus Asche wieder Leben. Der König aber wird seine Gier teuer bezahlen."
ChatGPT-4o_1.txt (line 32): So wurde sie bekannt als die "Tochter des Silbers", und wer sie einmal gesehen hatte, verg

In [56]:
CORPUS_DIR = Path(r"C:\Users\Sophia\Downloads\MA\CORPORA\Literary Fairytales")

QUOTE_MAP = str.maketrans({
    '«': '"', '»': '"',
    '„': '"', '“': '"', '”': '"',
    '‹': '"', '›': '"',
})
QUOTE_CHARS = list(map(chr, QUOTE_MAP.keys()))

READ_ENCODINGS = ["utf-8", "utf-16", "cp1252", "latin-1"]

def read_with_fallback(fp: Path) -> str:
    for enc in READ_ENCODINGS:
        try:
            return fp.read_text(encoding=enc)
        except UnicodeDecodeError:
            continue
    return fp.read_text(encoding="utf-8", errors="replace")

changed_files = 0
total_replacements = 0

for fp in CORPUS_DIR.rglob("*.txt"):
    text = read_with_fallback(fp)
    replacements_here = sum(text.count(ch) for ch in QUOTE_CHARS)
    if replacements_here:
        new_text = text.translate(QUOTE_MAP)
        fp.write_text(new_text, encoding="utf-8")  # vereinheitlicht auf UTF-8
        changed_files += 1
        total_replacements += replacements_here

print(f"Geänderte Dateien: {changed_files}, Ersetzungen gesamt: {total_replacements}")

Geänderte Dateien: 13, Ersetzungen gesamt: 2916


3. Description Corpora | Word and Token Counts

In [54]:
corpora = {
    "German_FFT": r"C:\Users\Sophia\Downloads\MA\CORPORA\German FFT",
    "ChatGPT_4": r"C:\Users\Sophia\Downloads\MA\CORPORA\GPT-4o",
    "ChatGPT_5": r"C:\Users\Sophia\Downloads\MA\CORPORA\GPT-5",
}


word_like = re.compile(r"[A-Za-z0-9]")  

def count_tokens_and_words(text: str):
    toks = word_tokenize(text)                
    token_count = len(toks)                   
    word_count = sum(bool(word_like.search(t)) and ('_' not in t) for t in toks)
    return token_count, word_count

def corpus_stats(folder):
    folder = Path(folder)
    token_counts, word_counts = [], []
    files = sorted(folder.glob("*.txt"))

    for fp in files:
        txt = fp.read_text(encoding="utf-8")
        tok_c, w_c = count_tokens_and_words(txt)
        token_counts.append(tok_c)
        word_counts.append(w_c)

    n_docs = len(files)
    total_tokens = sum(token_counts)
    total_words = sum(word_counts)

    mean_tokens = stats.mean(token_counts) if token_counts else 0
    mean_words  = stats.mean(word_counts)  if word_counts  else 0
    median_words = stats.median(word_counts) if word_counts else 0

    if isinstance(median_words, float) and median_words.is_integer():
        median_words_display = int(median_words)
    else:
        median_words_display = median_words

    return {
        "Number of Documents": n_docs,
        "Token Count (sum)": total_tokens,
        "Word Count (sum)": total_words,
        "Mean Token Count / Doc": round(mean_tokens, 2),
        "Mean Word Count / Doc": round(mean_words, 2),
        "Median Word Count / Doc": median_words_display,
    }

# table
rows = []
for name, path in corpora.items():
    rows.append({"Corpus": name, **corpus_stats(path)})

df = pd.DataFrame(rows).set_index("Corpus")
display(df)

,Number of Documents,Token Count (sum),Word Count (sum),Mean Token Count / Doc,Mean Word Count / Doc,Median Word Count / Doc
Corpus,,,,,,
German_FFT,123,241159,200782,1960.64,1632.37,1437.0
ChatGPT_4,100,139923,114701,1399.23,1147.01,1152.0
ChatGPT_5,100,152570,126151,1525.70,1261.51,1266.5


In [55]:
df.to_csv(r"C:\Users\Sophia\Downloads\MA\TABLES\corpora_summary.csv", encoding="utf-8", index=True)

Renaming Corpus files for R Studio

In [60]:
folder = Path(r"C:\Users\Sophia\Downloads\MA\R_Studio\Kunstmärchen")

prefix = "Kunstmärchen_"

for file in folder.glob("*.txt"):
    if not file.name.startswith(prefix):
        new_name = prefix + file.name
        new_path = file.with_name(new_name)
        file.rename(new_path)
        print(f"Renamed: {file.name} → {new_name}")

Renamed: Goethe_Das Märchen.txt → Kunstmärchen_Goethe_Das Märchen.txt
Renamed: Hauff_Das Märchen vom falschen Prinzen.txt → Kunstmärchen_Hauff_Das Märchen vom falschen Prinzen.txt
Renamed: Hauff_Die Errettung Fatmes.txt → Kunstmärchen_Hauff_Die Errettung Fatmes.txt
Renamed: Hauff_Die Geschichte von dem Gespensterschiff.txt → Kunstmärchen_Hauff_Die Geschichte von dem Gespensterschiff.txt
Renamed: Hauff_Die Geschichte von dem kleinen Muck.txt → Kunstmärchen_Hauff_Die Geschichte von dem kleinen Muck.txt
Renamed: Hauff_Die Geschichte von der abgehauenen Hand.txt → Kunstmärchen_Hauff_Die Geschichte von der abgehauenen Hand.txt
Renamed: Hauff_Kalif Storch.txt → Kunstmärchen_Hauff_Kalif Storch.txt
Renamed: Hoffmann_Der goldene Topf.txt → Kunstmärchen_Hoffmann_Der goldene Topf.txt
Renamed: Hoffmann_Die Königsbraut.txt → Kunstmärchen_Hoffmann_Die Königsbraut.txt
Renamed: Hoffmann_Meister Floh.txt → Kunstmärchen_Hoffmann_Meister Floh.txt
Renamed: Tieck_Der blonde Eckbert.txt → Kunstmärchen_Tieck